# Install Libraries


In [ ]:
!pip install -q -U langchain transformers bitsandbytes accelerate
!pip install -U langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 18.6 MB/s eta 0:00:00


# Import Libraries

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,PreTrainedTokenizerFast
import re
import csv
import torch
import os
import json


# Authenticate with Hugging Face


In [ ]:
#
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `notebook` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when 

# Load the LLM


In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    # bnb_4bit_compute_dtype=torch.float16,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_use_double_quant=True,
)

tokenizer = PreTrainedTokenizerFast.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
model_4bit = AutoModelForCausalLM.from_pretrained( "meta-llama/Llama-3.1-8B-Instruct", device_map="auto",quantization_config=quantization_config, )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]


**Load the LLM**

In [ ]:
pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=4096,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline_inst)

# Split commentary into chunks for easier processing


In [ ]:
def split_commentary_from_file(file_path, num_parts=5):
    # Read the commentary from the text file
    with open(file_path, "r") as file:
        commentary_text = file.read()

    # Split the commentary into lines
    lines = commentary_text.strip().split("\n")

    # Calculate the number of lines per part
    total_lines = len(lines)
    lines_per_part = total_lines // num_parts

    # Split the lines into parts
    parts = []
    for i in range(num_parts):
        start = i * lines_per_part
        end = (i + 1) * lines_per_part if i < num_parts - 1 else total_lines
        part = "\n".join(lines[start:end])
        parts.append(part)

    return parts



In [ ]:
# Path to the text file containing the commentary
file_path = "/content/fullCommentary.txt"  # Replace with the actual path to your file

# Split the commentary into 5 parts
num_parts = 10
parts = split_commentary_from_file(file_path, num_parts)

# Print the parts
for i, part in enumerate(parts, 1):
    print(f"Part {i}:\n{part}\n")

Part 1:
00:00.000 --> 00:05.520
at eight days ago. Portugal regularly used a 4-4-2 formation in qualifying,

00:05.520 --> 00:11.360
whether Gaydash plays just behind Ronaldo or alongside him, we will just have to wait and see.

00:15.360 --> 00:21.280
Portugal have played in more World Cups with Cristiano Ronaldo than without.

00:21.280 --> 00:26.560
This is their fourth with their talisman. It's their seventh World Cup in all.

00:27.200 --> 00:30.160
Gianluca Rocchi from Italy is in charge.

00:31.040 --> 00:36.080
Two Italian assistants, Raiusi Sato from Japan is the fourth official.

00:38.400 --> 00:46.080
And back in Moscow, there is the lineup of VAR officials led by Massimiliano Eratti from Italy.

00:46.080 --> 01:00.320
David De Gea was never present in qualifying. Many of this Spanish team we know would have

01:00.320 --> 01:05.840
started no matter who the coach is. Sergio Puskats is arguably Spain's most important player.

01:05.840 --> 01:11.520
Andres Iniesta scored t

# Question format for the LLM

In [ ]:
Question='''
End of Commentary.
Question:
What type of events exist in each segment of the commentary? For each time segment, choose the event type from one of the following:

-Direct free-kick

-Penalty

-Shots off target

-Yellow card

-Throw-in

-Goal

-Ball out of play

-Indirect free-kick

-Kick-off

-Foul

-Shots on target

-Offside

-Corner

-Red card

-Yellow->red card

-No event existed in the given time
'''

# Template

In [ ]:
template2 = """<s>[INST] You are a soccer commentary event analyzer. Your job is to decide if a commentary contains any event.
Answer the question below from the context provided, and format your output as follows:
 "time - (event type)"
Provide your response strictly in the above format.

**Instructions:**
1. Only use the time ranges explicitly mentioned in the given commentary. Do not create or infer time ranges that are not provided.
2. For each time range, analyze if an event occurred based on the provided text.
3. If no clear event is described during the time range, respond with "No event existed in the given time."
4. Ensure your response is concise and adheres strictly to the required format.

{question} [/INST]</s>
"""


In [ ]:
def generate_response(question):
  prompt = PromptTemplate(template=template2, input_variables=["question","context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question})
  return response

# Helper function to parse the LLm response into desired format

In [ ]:
def parse_llm_response(response):
    # Extract the answer part (after [/INST] </s>)
    answer_part = response.split("[/INST]</s>")[-1].strip()
    print(answer_part)

    # Define the regex patterns
    pattern1 = r"\* (\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}\.\d{3}) - (.*)"
    pattern2 = r"(\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}\.\d{3}) - (.*)"
    pattern3 = r"(\d{2}:\d{2}\.\d{3}) - (\d{2}:\d{2}\.\d{3}): (.*)"
    pattern4 = r"(\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}\.\d{3})\s+(.*)"
    pattern5 = r"(\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}\.\d{3}) - \((.*)\)"
    pattern6 = r"\* (\d{2}:\d{2}\.\d{3}) - (\d{2}:\d{2}\.\d{3}) - \((.*)\)"
    pattern7 = r"(\d{2}:\d{2}\.\d{3})-(\d{2}:\d{2}\.\d{3}) - (.*)"
    pattern8 = r"(\d{2}:\d{2}\.\d{3}) - \((.*)\)"
    pattern9 = r"(\d{2}:\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}:\d{2}\.\d{3})\s+(.*)"
    pattern10 = r"(\d{2}:\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}:\d{2}\.\d{3}) - (.*)"
    pattern11 = r"(\d{2}:\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}:\d{2}\.\d{3}) - No event existed in the given time"

    # Combine all patterns into a single regex
    combined_pattern = (
        f"{pattern1}|{pattern2}|{pattern3}|{pattern4}|{pattern5}|{pattern6}|"
        f"{pattern7}|{pattern8}|{pattern9}|{pattern10}|{pattern11}"
    )

    # Find all matches in the answer part
    matches = re.findall(combined_pattern, answer_part)
    print(matches)

    # Store parsed events
    events = []
    for match in matches:
        # Check which pattern matched
        if match[0]:  # Pattern 1 matched
            start_time = match[0].strip()
            end_time = match[1].strip()
            event_type = match[2].strip()
        elif match[3]:  # Pattern 2 matched
            start_time = match[3].strip()
            end_time = match[4].strip()
            event_type = match[5].strip()
        elif match[6]:  # Pattern 3 matched
            start_time = match[6].strip()
            end_time = match[7].strip()
            event_type = match[8].strip()
        elif match[9]:  # Pattern 4 matched
            start_time = match[9].strip()
            end_time = match[10].strip()
            event_type = match[11].strip()
        elif match[12]:  # Pattern 5 matched
            start_time = match[12].strip()
            end_time = match[13].strip()
            event_type = match[14].strip()
        elif match[15]:  # Pattern 6 matched
            start_time = match[15].strip()
            end_time = match[16].strip()
            event_type = match[17].strip()
        elif match[18]:  # Pattern 7 matched
            start_time = match[18].strip()
            end_time = match[19].strip()
            event_type = match[20].strip()
        elif match[21]:  # Pattern 8 matched
            start_time = match[21].strip()
            end_time = ""  # No end time provided
            event_type = match[22].strip()
        elif match[23]:  # Pattern 9 matched
            start_time = match[23].strip()
            end_time = match[24].strip()
            event_type = match[25].strip()
        elif match[26]:  # Pattern 10 matched
            start_time = match[26].strip()
            end_time = match[27].strip()
            event_type = match[28].strip()
        else:  # Pattern 11 matched
            start_time = match[29].strip()
            end_time = match[30].strip()
            event_type = "No event existed in the given time"

        # Add the parsed event to the list
        events.append({
            "start_time": start_time,
            "end_time": end_time,
            "event_type": event_type
        })

    return events

def save_to_csv(events, file_path):
    # Define the CSV column headers
    headers = ["Start Time", "End Time", "Event Type"]

    with open(file_path, "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=headers)

        # Write the headers
        writer.writeheader()

        # Write the events
        for event in events:
            writer.writerow({
                "Start Time": event["start_time"],
                "End Time": event["end_time"],
                "Event Type": event["event_type"]
            })



**Helper function to convert the parsed output into json structure**

In [ ]:

def convert_csv_to_json(csv_file_path, output_json_file):
    """
    Converts a CSV file with match events into a JSON annotation file.

    Args:
        csv_file_path (str): Path to the input CSV file.
        output_json_file (str): Path to save the output JSON file.
    """
    # Define the allowed event types
    allowed_events = {
        "Direct free-kick", "Penalty", "Shots off target", "Yellow card",
        "Throw-in", "Goal", "Ball out of play", "Indirect free-kick",
        "Kick-off", "Foul", "Shots on target", "Offside", "Corner",
        "Red card", "Yellow->red card", "Clearance", "Substitution"
    }

    # Initialize the JSON structure
    json_data = {
        "version": "1.1",
        "date": "",  # Leave date empty
        "videos": [
            {
                "annotations": [],
                "path": "",  # Leave path empty
                "input_type": "video"
            }
        ]
    }

    # Read the CSV file and process rows
    with open(csv_file_path, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            event_type = row["Event Type"].strip()
            if event_type[1:] in allowed_events  or event_type in allowed_events or event_type[:-5] in allowed_events:
                # Convert time format to match the required JSON structure
                start_time = row["Start Time"]
                game_time = f"1 - {start_time}"

                # Add the annotation
                annotation = {
                    "gameTime": game_time,
                    "label": event_type,
                    "position": 0,  # Default value, adjust as needed
                    "team": "not applicable",  # Default value, adjust as needed
                    "visibility": "visible"  # Default value, adjust as needed
                }
                json_data["videos"][0]["annotations"].append(annotation)

    # Save the JSON data to a file
    with open(output_json_file, mode="w") as file:
        json.dump(json_data, file, indent=4)

    print(f"JSON file saved as {output_json_file}")


# Example usage
# json_files_directory='/content/content/csvOut'
# i=1
# for filename in os.listdir(json_files_directory):
#     if filename.endswith(".csv"):
#         file_path = os.path.join(json_files_directory, filename)
#         csv_file = file_path  # Replace with your CSV file path
#         output_json_file = f"/content/json6/annotations_{i}.json"  # Replace with your desired output JSON file path
#         convert_csv_to_json(csv_file, output_json_file)
#         i+=1

JSON file saved as /content/json6/annotations_1.json
JSON file saved as /content/json6/annotations_2.json
JSON file saved as /content/json6/annotations_3.json
JSON file saved as /content/json6/annotations_4.json
JSON file saved as /content/json6/annotations_5.json
JSON file saved as /content/json6/annotations_6.json
JSON file saved as /content/json6/annotations_7.json
JSON file saved as /content/json6/annotations_8.json
JSON file saved as /content/json6/annotations_9.json
JSON file saved as /content/json6/annotations_10.json


In [ ]:
!mkdir -p content/csvOut
!mkdir -p content/json


# Run the LLm inference on all the chunks of the commentary
**each response gets parsed and formatted**

In [ ]:
for i, comment in enumerate(parts, 1):
    torch.cuda.empty_cache()
    Q_with_commentary = comment + "\n\n" + Question
    res = generate_response(Q_with_commentary)
    # Parse the response
    parsed_events = parse_llm_response(res)
    # Save the parsed events to a CSV file
    save_to_csv(parsed_events, f"content//csvOut//events_{i}.csv")
    output_json_file = f"json//annotations_{i}.json"
   # Convert a CSV file to JSON
    convert_csv_to_json(f"csvOut//events_{i}.csv",output_json_file)



01:00:58.000 --> 01:01:00.000 - No event existed in the given time
01:01:01.760 --> 01:01:03.760 - No event existed in the given time
01:01:05.760 --> 01:01:13.440 - No event existed in the given time
01:01:16.480 --> 01:01:20.960 - No event existed in the given time
01:01:23.520 --> 01:01:25.520 - No event existed in the given time
01:01:26.160 --> 01:01:28.560 - No event existed in the given time
01:01:29.280 --> 01:01:31.280 - No event existed in the given time
01:01:33.200 --> 01:01:35.200 - No event existed in the given time
01:01:36.960 --> 01:01:38.960 - No event existed in the given time
01:01:39.520 --> 01:01:42.960 - Shots on target
01:01:45.120 --> 01:01:49.440 - No event existed in the given time
01:01:51.120 --> 01:01:53.120 - No event existed in the given time
01:01:53.200 --> 01:01:55.600 - No event existed in the given time
01:01:58.720 --> 01:02:08.720 - No event existed in the given time
01:02:11.120 --> 01:02:13.120 - No event existed in the given time
01:02:14.000 -

# For each chunk , we had a json response ,the following code is to merge all of them and get the final coomentary predictions in the format desired

In [ ]:
json_files_directory = "/content/json"  # Replace with the path to your JSON files

# Initialize the final combined JSON structure
combined_json = {
    "version": "1.1",
    "date": "",  # Leave date empty
    "videos": [
        {
            "annotations": [],
            "path": "uploads/fullmatch.mp4",                  # change with your video path
            "input_type": "video"
        }
    ]
}

# Loop through all JSON files in the directory
for filename in os.listdir(json_files_directory):
    if filename.endswith(".json"):
        file_path = os.path.join(json_files_directory, filename)
        with open(file_path, mode="r") as file:
            data = json.load(file)
            # Extract annotations from the current file
            annotations = data["videos"][0]["annotations"]
            # Add them to the combined annotations list
            combined_json["videos"][0]["annotations"].extend(annotations)

# Save the combined JSON to a new file
output_json_file = "standard.json"
with open(output_json_file, mode="w") as file:
    json.dump(combined_json, file, indent=4)

print(f"Combined JSON file saved as {output_json_file}")

Combined JSON file saved as combined_annotations.json
